In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
import time
from random import *
from collections import deque
from sklearn import datasets, linear_model
from scipy import stats

class vertex:
    # this class creates a vertex within the SG
    def __init__(self,xpos, ypos):
        self.r = [xpos, ypos]
        self.ns = [] #this is list of the neighbors of the vertex
        self.d = 0 #this is the direction that the edge is pointing in the rotor router algorithm
        self.occ = False #this boolean property determines whether of or not the vertex is occupied by a particle
        self.level = max(abs(self.r[0]),abs(self.r[1])) # this is the "sphere" that the vertex lies on
        self.col = int(round(self.r[0]))
        
    def __eq__(self, other):
        return self.r == other.r
    
    def __hash__(self): 
        return hash((self.r[0], self.r[1]))


def generate(n):
    G = nx.Graph()
    myvert = vertex(0,0)
    nx.add_star(G, [vertex(*r) for r in [(0,0), (1,1), (-1,1), (1,-1), (-1,-1)]]) #Tuples

    for i in range(n-1):
        N = 2*3**(i)
        H = G.copy()
        for (Nx, Ny) in [(N,N), (N,-N), (-N,N), (-N,-N)]:
            H_copy = nx.relabel_nodes(H, lambda v:vertex(v.r[0] + Nx, v.r[1] + Ny))
            G.add_edges_from(H_copy.edges)
    
    #Set Graph Attributes
    nx.set_node_attributes(G, False,'occupied')
    for node in (G.nodes):
        G.nodes[node]['d'] = randint(0, 3)
        
    
    return G

def plotting(G,n, launched):
    pos = {v:v.r for v in G}
    
    occupied_nodes = [n for n, occ in G.nodes.data('occupied') if occ]
    empty_nodes = [n for n, occ in G.nodes.data('occupied') if not occ]
    print(occupied_nodes)
    
    # Get current size
    fig_size = plt.rcParams["figure.figsize"]

    # Set figure width to 12 and height to 9
    fig_size[0] = 12
    fig_size[1] = 9
    plt.rcParams["figure.figsize"] = fig_size
    plt.title('Particles Launched: '+ str(launched))
    
    nx.draw(G, pos=pos, nodelist=empty_nodes, node_color='r', node_size = 300/(n**2))
    nx.draw(G, pos=pos, nodelist=occupied_nodes, node_color='b', node_size = 300/(n**2))
    
    plt.savefig('rotor-router_particle(' + str(launched) + ').png')
    
    plt.show()
    
    
    

def rotor_router(G):
    
    myvert = vertex(0,0)  #Initialize it to the (0,0)
    
    while G.nodes[myvert]['occupied']:
        #print(G.nodes[myvert]['d'])
        neighbors = list(G.adj[myvert].keys()) #Neighbors of the node we are looking at
        print(len(neighbors))
        d_prev = G.nodes[myvert]['d']   #Where the node points
        G.nodes[myvert]['d'] = (d_prev + 1)%len(neighbors)
        myvert = neighbors[(d_prev + 1)%len(neighbors)] 
    
    G.nodes[myvert]['occupied'] = True


def abelian_sandpile(G, piles):
    origin = vertex(0,0)
    nx.set_node_attributes(G, 0, 'grain')
    G.nodes[origin]['grain'] = piles
    queue = deque([origin])
    
    radius = 0
    
    while (len(queue) > 0):
        myvert = queue.popleft()
        highest_pile = G.nodes[myvert]['grain']
        neighbors = list(G.adj[myvert].keys())
        G.nodes[myvert]['grain'] = highest_pile%(len(neighbors))
        for node in neighbors:
            G.nodes[node]['grain'] +=  int(highest_pile/len(neighbors))
            if np.absolute(np.amax(node.r)) > radius:
                radius = np.absolute(np.amax(node.r))
            if node not in queue:
                if G.nodes[node]['grain'] >= len(list(G.adj[node].keys())):
                    queue.append(node)
    plot_sandpile(G, piles)  #Uncomment to plot
    return radius

        
def plot_sandpile(G, launched):
    pos = {v:v.r for v in G}
    color = [[],[],[],[]]
    
    for node in G.nodes:
        color[G.nodes[node]['grain']].append(node)
    
    # Get current size
    fig_size = plt.rcParams["figure.figsize"]

    # Set figure width to 12 and height to 9
    fig_size[0] = 20
    fig_size[1] = 15
    plt.rcParams["figure.figsize"] = fig_size
    plt.title('Particles Launched: '+ str(launched))
        
    nx.draw(G, pos=pos, nodelist=color[0], node_color='magenta', node_size = 15)
    nx.draw(G, pos=pos, nodelist=color[1], node_color='green', node_size = 15)
    nx.draw(G, pos=pos, nodelist=color[2], node_color='cyan', node_size = 15)
    nx.draw(G, pos=pos, nodelist=color[3], node_color='blue', node_size = 15)
    
    import matplotlib.patches as mpatches
    mag = mpatches.Patch(color='magenta', label='0')
    green = mpatches.Patch(color='green', label='1')
    cyan = mpatches.Patch(color='cyan', label='2')
    blue = mpatches.Patch(color='blue', label='3')
    plt.legend(handles=[mag, green, cyan, blue], frameon = False)
    
    plt.savefig('abelian_sandpile('+ str(launched) + ').png')
    
    plt.show()
    

def cycles(particles, level, cycle=1): #Sets all nodes to unoccupied each cycle
    G = generate(level)
    for j in range(cycle):
        for i in range(particles):
            rotor_router(G)
            plotting(G, level, i+1)
        nx.set_node_attributes(G, False,'occupied')


def cycles2(particles, level, cycle=1): #Regenerates Graph each cycle
    for j in range(cycle):
        G = generate(level)
        for i in range(particles):
            rotor_router(G)
            plotting(G, level, i+1)
        G.clear()
        #nx.set_node_attributes(G, False,'occupied')
        
    
def generate_radii(start, upto):
    radii = []
    G = generate(5)
    for j in range(start,upto,4):
        radii.append(abelian_sandpile(G,j))
    return radii
    
def radiusPlot(radii):
    mass = [i for i in range(1,len(radii)+1)]
    dh = 1 / (np.log(5)/np.log(3))
    for j in range(len(radii)):
        radii[j] = radii[j]/(mass[j]**dh)
    radii = np.log(radii)
    mass = np.log(mass)
    #slope, intercept = np.polyfit(mass,radii, 1)
    #yp = np.polyval([slope, intercept], mass)
    plt.plot(mass, radii)
    plt.title("Log-Log Plot")
    plt.xlabel('4m')
    plt.ylabel('r(m)/(m^(1/dh))')
    fig_size = plt.rcParams["figure.figsize"]
    plt.show()
    
#Data for radii from 1 -> 900 inclusive
#radii = [1, 2, 2, 2, 3, 4, 4, 4, 5, 6, 6, 6, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 10, 10, 10, 11, 12, 12, 12, 13, 13, 13, 14, 14, 14, 14, 14, 14, 14, 14, 14, 15, 16, 16, 16, 17, 18, 18, 18, 19, 19, 19, 20, 20, 20, 20, 20, 20, 20, 20, 20, 21, 21, 21, 22, 22, 22, 22, 22, 22, 22, 22, 22, 23, 23, 23, 24, 24, 24, 24, 24, 24, 24, 24, 24, 25, 25, 25, 25, 25, 25, 25, 25, 25, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 27, 28, 28, 28, 29, 30, 30, 30, 31, 31, 31, 32, 32, 32, 32, 32, 32, 32, 32, 32, 33, 34, 34, 34, 35, 36, 36, 36, 37, 37, 37, 38, 38, 38, 38, 38, 38, 38, 38, 38, 39, 39, 39, 40, 40, 40, 40, 40, 40, 40, 40, 40, 41, 41, 41, 42, 42, 42, 42, 42, 42, 42, 42, 42, 43, 43, 43, 43, 43, 43, 43, 43, 43, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 45]

#Rotor-Router and Sandpile
#Odometer function = for each vertex, count how many number of times it topples

#IDLA
#In and out radii
# Late function ( if pi*r(j)^2 > j: early else:late)
#VPN


cycles(100, 3)

#cycles(20,3,1)
#891


4
4
4
4
4
2
4
2
4
2
4
2
4
2
4
2
4
2
4
2
4
2
4
2
4
2
4
4
2
4
2
4
2
4
2
4
4
2
4
2
4
4
2
4
4
2
4
2
4
2
4
2
4
2
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
2
4
2
4
2
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
2
4
2
4
2
4
2
4
2
4
2
4
1
4
2
4
2
4
2
4
2
4
2
4
2
4
1
4
1
4
2
4
2
4
2
4
2
4
1
4
1
4
2
4
2
4
1
4
1
4
2
4
2
4
2
4
2
4
2
4
1
4
1
4
2
4
2
4
2
4
2
4
1
4
1
4
2
4
2
4
2
4
2
4
2
4
2
4
1
4
1
4
2
4
2
4
2
4
2
4
1
4
1
4
2
4
2
4
2
4
2
4
2
4
1
4
1
4
2
4
2
4
1
4
1
4
2
4
2
4
2
4
2
4
1
4
1
4
2
4
2
4
1
4
1
4
2
4
2
4
2
4
2
4
4
2
4
1
4
1
4
2
4
4
2
4
2
4
4
2
4
1
4
1
4
2
4
4
2
4
2
4
2
4
2
4
2
4
1
4
1
4
2
4
2
4
2
4
2
4
1
4
1
4
2
4
2
4
2
4
2
4
2
4
1
4
1
4
2
4
2
4
1
4
1
4
2
4
2
4
2
4
2
4
1
4
1
4
2
4
2
4
1
4
1
4
2
4
2
4
2
4
2
4
4
2
4
1
4
1
4
2
4
4
2
4
2
4
2
4
1
4
1
4
2
4
2
4
1
4
1
4
2
4
4
2
4
2
4
2
4
2
4
4
2
4
2
4
1
4
1
4
2
4
2
4
2
4
2
4
2
4
2
4
2
4
2
4
1
4
1
4
2
4
2
4
1
4
1
4
2
4
2
4
2
4
1
4
1
4
2
4
2
4
1
4
1
4
2
4
2
4
2
4
2
4
4
2
4
1
4
1
4
2
4
4
2
4
2
4
1
4
1
4
2
4
2
4
1
4
1
4
2
4
2
4
2
4
2
4
2
4
2
4
2
4
4
2
4
1
4
1
4
2
4
4
2
4
1
4
1


In [1]:
import networkx as nx
nx.__version__

'2.0'

In [ ]:
print (generate_radii(904,1001))